# Simple knodle trainer

In [1]:
import logging

import numpy as np
import joblib 
import re

import pandas as pd
import torch   

import torch.nn as nn
from torch.optim import SGD
from torch.utils.data import TensorDataset

from transformers import BertTokenizerFast, AdamW, BertConfig, BertForSequenceClassification, AutoModel, BertModel, BertConfig, AutoConfig
from knodle.trainer import AutoTrainer, AutoConfig, baseline, MajorityVoteTrainer, multi_trainer
#from knodle.trainer.baseline.config import MajorityConfig
from config import MajorityConfig
from knodle.transformation.majority import input_to_majority_vote_input
from knodle.transformation.torch_input import input_labels_to_tensordataset
from knodle.trainer.trainer import BaseTrainer
from knodle.transformation.filter import filter_probability_threshold

2023-01-27 16:02:00,380 root         INFO     Initalized logger


In [2]:
# load the data

x_train = joblib.load('./train_embeddings_torch_540.joblib')
z_train = joblib.load('./z_torch_matrix.joblib')
z_train = np.array(z_train)
#t_train = joblib.load('./t_torch_matrix.joblib')
t_train = joblib.load('./t_matrix_v02.joblib')    # dim: 74 * 3
t_train = np.array(t_train)
x_test = joblib.load('./test_embeddings.joblib')
y_test = joblib.load('./test_data_labels.joblib')

In [3]:
x_train = x_train.type(torch.IntTensor)

In [4]:
print(x_train.size())
print(z_train.shape)
print(len(z_train), len(z_train[0]))
#print(t_train.shape)
print(len(t_train), len(t_train[0]))
print(x_test.size())
print(y_test.size())

torch.Size([540, 768])
(4627, 74)
4627 74
74 3
torch.Size([195, 768])
torch.Size([8411, 3])


In [5]:
z_train = z_train[:540, :]
#z_train = torch.stack(z_train, dim=0).numpy()
y_test = y_test[:195, :]

In [6]:
## convert tensors to numpy
#z_train_np = z_train.cpu().detach().numpy()
###t_train_np = t_train.cpu().detach().numpy()
#z_train_np.shape

In [7]:
x_train = TensorDataset(x_train)
x_test = TensorDataset(x_test)

In [8]:
x_train

In [9]:
model_name = "bert-base-uncased"
model = BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
custom_model_config = MajorityConfig(
    optimizer=AdamW,
    lr=1e-4,
    batch_size=2,
    epochs=2,
    filter_non_labelled=True,
    output_classes = 3
)

print(custom_model_config.__dict__)

trainer = MajorityVoteTrainer(
    model=model,
    mapping_rules_labels_t=t_train,
    model_input_x=x_train,
    rule_matches_z=z_train,
    trainer_config=custom_model_config,
)

trainer.train()

2023-01-27 16:02:08,455 knodle.trainer.config INFO     The cache will be saved to C:\Users\stephanie\Uni\Data_Science_project\Code\cache folder
2023-01-27 16:02:08,456 knodle.trainer.config INFO     The trained models will be saved to the C:\Users\stephanie\Uni\Data_Science_project\Code\cache directory.
2023-01-27 16:02:08,462 knodle.trainer.config INFO     Model will be trained on cpu
2023-01-27 16:02:08,489 knodle.transformation.rule_reduction INFO     No filtering criteria ('max_rule' or 'min_coverage' for rule specified, returning the original rule matches.
C:\Users\stephanie\anaconda3\envs\ds_project\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
C:\Users\stephanie\anaconda3\envs\ds_project\lib\site-packages\knodle\transformation\majority.py:78:

{'seed': None, 'caching_suffix': '', 'caching_folder': 'C:\\Users\\stephanie\\Uni\\Data_Science_project\\Code\\cache', 'saved_models_dir': 'C:\\Users\\stephanie\\Uni\\Data_Science_project\\Code\\cache', 'criterion': <function cross_entropy_with_probs at 0x0000024093FD43A8>, 'lr': 0.0001, 'batch_size': 2, 'output_classes': 3, 'grad_clipping': None, 'device': device(type='cpu'), 'epochs': 2, 'optimizer': <class 'transformers.optimization.AdamW'>, 'class_weights': tensor([1., 1., 1.]), 'filter_non_labelled': True, 'other_class_id': None, 'evaluate_with_other_class': False, 'ids2labels': None, 'max_rules': None, 'min_coverage': None, 'drop_rules': False, 'use_probabilistic_labels': False, 'probability_threshold': None}


  0%|          | 0/146 [00:00<?, ?it/s]

RuntimeError: The expanded size of the tensor (768) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [2, 768].  Tensor sizes: [1, 512]

In [ ]:
#config-file:

#from knodle.trainer.auto_config import AutoConfig
#from knodle.trainer.config import BaseTrainerConfig


#@AutoConfig.register("majority")
#class MajorityConfig(BaseTrainerConfig):
#    def __init__(
#            self,
#            use_probabilistic_labels: bool = False,
#            **kwargs
#    ):
#        super().__init__(**kwargs)
#        self.use_probabilistic_labels = use_probabilistic_labels
#        self.probability_threshold = None